# Transformace příznaků
Úloha zaměřená na implementaci metod PCA a LDA pro transformaci příznaků. 

Výchozí motivace
- Cílem je provést dekorelaci příznaků a vybrat pouze ty nejvýznamnější

Chceme
- Snížit zátěž (výpočetní, časovou) vlastního klasifikačního procesu.
- Zvýšit úspěšnost klasifikace

## Data

In [95]:
import numpy as np
import usu

npzfile = np.load('data/data_12.npz')
npzfile.files


['testData', 'testRef', 'trainData', 'trainRef']

In [96]:
testData = npzfile['testData']
testRef = npzfile['testRef']

trainData = npzfile['trainData']
trainRef = npzfile['trainRef']

trainData.shape,trainRef.shape, testData.shape, testRef.shape

((1050, 5), (1050, 1), (850, 5), (850, 1))

### Výpočet úspěšnosti
$$ accuracy = \frac{\text{počet správně klasifikovaných objektů}}{\text{počet všech klasifikovaných  objektů}} $$

In [97]:
def accuracy(testRef, predRef):
    """
    vraci uspesnost v procentech
    """
    return np.sum(predRef == testRef) / testRef.shape[0] * 100


### Vzdálenostní funkce
V dané uloze implemetujeme jenom euklidovskou vzdálenost

#### Euklidovská vzdálenost (L2)
$$ d(x,z) = \sqrt{\sum_{i=0}^{Dim}{(x_i - z_i)^2}} $$

Při implementaci jde vynechat operaci druhé odmocniny, protože druhá odmocnina je monotónní rostoucí funkce. To znamená, že se mění jenom absolutní hodnoty vzdálenosti, ale pořadí se zachovává:
$$ d(x,z) = \sum_{i=0}^{Dim}{(x_i - z_i)^2} $$

In [98]:
def euclidian_distance (testItem, trainData):
    """
    vypocet vzdalenosti jendoho testovaciho vzorku ke vsem trenovacim datum
    """

    #using Broadcasting
    diff = trainData - testItem  # Shape: (n_samples, n_features)

    squared_diff = diff ** 2  # Shape: (n_samples, n_features)

    #sum feature-wise
    distances = np.sum(squared_diff, axis=1, keepdims=True)  # Shape: (n_samples, 1)

    # POZNÁMKA: Vynecháváme sqrt() protože pořadí zůstává stejné
    # sqrt je monotónní rostoucí funkce → nepotřebujeme ji pro porovnávání

    return distances

euclidian_distance(testData[0],trainData)

array([[38.54881076],
       [54.30823974],
       [56.90800423],
       ...,
       [39.14515371],
       [20.25885791],
       [17.23315459]])

In [99]:
def getPrediction(trainData, trainRef, testData):

    #pomoci funkce euclidian_distance
    predRef = np.zeros([testData.shape[0], 1])

    for i in range(testData.shape[0]):
        distances = euclidian_distance(testData[i], trainData)
        nearest_idx = np.argmin(distances)
        predRef[i] = trainRef[nearest_idx]

    return predRef


In [100]:
predRef = getPrediction(trainData, trainRef,testData)
print(f"acc : {accuracy(testRef,predRef):.2f}")


acc : 89.65


## Metody transformace příznaků
### Transformace na základě dekorelace (a následná redukce): PCA
- Z dat $X$ určíme kovariační matici $\Sigma_x$
- Vypočteme její vlastní vektory $e$ a sestavíme z nich matici $E$
- Pak transformovaná data:

$$X_{tr} = X (E_{0:f})^T$$
kde f je počet příznaků
- **Kovariační matice transformovaných dat bude diagonální**
- **Data v novém souřadném systému budou dekorelovaná**


### Transformace s ohledem na co největší diskriminativnost: LDA
Vypočteme matici vlastních vektorů $E$ z matice určené součinem $ \Sigma_{bc} \Sigma_{wc}^{-1} $

kde 
- kovarianční matice spočítaná ze středních hodnot tříd: $$\Sigma_{bc} = \frac{1}{N} \sum_{class=0}^{C} N_{class} (\mu_{class} - \mu)^T (\mu_{class} - \mu)$$ 
- průměrná kovarianční matice tříd: $$\Sigma_{wc} = \frac{1}{N} \sum_{class=0}^{C}{N_{class} \Sigma_{class}}$$ 


kde N je počet prvků a C je počet tříd

- Pak transformovaná data:

$$X_{tr} = X (E_{0:f})^T$$

In [101]:
class Transform:
    """
    Třída pro transformaci příznaků pomocí PCA a LDA.

    Detailní dokumentace v souboru: PCA_LDA_DOKUMENTACE.md
    """

    def __init__(self, trainData, trainRef, testData, testRef):
        self.trainData = trainData
        self.testData = testData
        self.trainRef = trainRef
        self.testRef = testRef
    
    def pca(self, data, nFeautures=1):
        """
        transformuje data pca transformacni matici
        """
        #
        #kovariacni matice
        cov_matrix = np.cov(self.trainData, rowvar=False)

        eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
        eigenvalues = np.real(eigenvalues)
        eigenvectors = np.real(eigenvectors)

        #sorting eigenvalues and eigenvectors
        idx = np.argsort(eigenvalues)[::-1]
        eigenvectors = eigenvectors[:, idx]

        #vyber top K komponent
        transformation_matrix = eigenvectors[:, :nFeautures]

        #transformace
        transformed_data = data @ transformation_matrix
        return transformed_data

    def lda(self, data, nFeautures=1):
        """
        transformuje data lda transformacni matici
        """
        classes = np.unique(self.trainRef)
        n_classes = len(classes)
        n_train_samples = self.trainData.shape[0]
        n_features = data.shape[1]

        #globalni stred
        global_mean = np.mean(self.trainData, axis=0)

        #between-class scatter matrix
        S_between = np.zeros((n_features, n_features))
        for c in classes:
            class_data = self.trainData[self.trainRef.flatten() == c]
            n_c = class_data.shape[0]
            class_mean = np.mean(class_data, axis=0)
            mean_diff = (class_mean - global_mean).reshape(-1, 1)
            S_between += n_c * (mean_diff @ mean_diff.T)
        S_between /= n_train_samples

        #within-class scatter matrix
        S_within = np.zeros((n_features, n_features))
        for c in classes:
            class_data = self.trainData[self.trainRef.flatten() == c]
            n_c = class_data.shape[0]
            class_cov = np.cov(class_data, rowvar=False)
            S_within += n_c * class_cov
        S_within /= n_train_samples

        #regularizace pro stabilitu
        S_within_reg = S_within + 1e-6 * np.eye(n_features)
        S_within_inv = np.linalg.inv(S_within_reg)
        matrix = S_within_inv @ S_between

        #eigenvalue problem
        eigenvalues, eigenvectors = np.linalg.eig(matrix)

        #serazeni a vyber komponent (max C-1)
        idx = np.argsort(np.abs(eigenvalues))[::-1]
        eigenvectors = eigenvectors[:, idx]
        max_components = min(nFeautures, n_classes - 1)
        transformation_matrix = eigenvectors[:, :max_components].real

        #transformace
        transformed_data = data @ transformation_matrix

        return transformed_data


In [102]:
transformation = Transform(trainData, trainRef, testData, testRef)


In [103]:
#pca
for dim in range(1,np.size(trainData,1)+1):
    trainDataT = transformation.pca(trainData, dim)
    testDataT = transformation.pca(testData, dim)
    
    predRef = getPrediction(trainDataT,trainRef,testDataT)
    print(f"priznaky: {dim} -> acc : {accuracy(testRef,predRef):.2f} %")

priznaky: 1 -> acc : 66.24 %
priznaky: 2 -> acc : 84.82 %
priznaky: 3 -> acc : 89.06 %
priznaky: 4 -> acc : 88.82 %
priznaky: 5 -> acc : 89.65 %


In [104]:
#lda
for dim in range(1,np.size(trainData,1)+1):
    trainDataT = transformation.lda(trainData, dim)
    testDataT = transformation.lda(testData, dim)
    
    predRef = getPrediction(trainDataT,trainRef,testDataT)
    print(f"priznaky: {dim} -> acc : {accuracy(testRef,predRef):.2f} %")
  

priznaky: 1 -> acc : 74.82 %
priznaky: 2 -> acc : 82.59 %
priznaky: 3 -> acc : 89.06 %
priznaky: 4 -> acc : 89.06 %
priznaky: 5 -> acc : 89.06 %


In [105]:
# Závěr:
# Kdy použít kterou metodu?
#obe metody hledaji optimalni rotaci souradnaho systemu, ale s odlisnymi cili:
# PCA: max variance (ignoruje tridy)
# LDA: max separace (využiva tridy)


# PCA - Komprese a explorativni analýza:
#  Nemusime mit pojmenovana data (unsupervised)
#  Komprese dat, odstraneni šumu
#  Maximalni variance - odhali strukturu dat
#  Negarantuje separaci trid (ignoruje labels)
#  Použiti: dimenzionalita, rychlost, vizualizace

# LDA - Klasifikace a separace:
#  Potrebujeme pojmenovana data (supervised)
#  Maximalni separace mezi tridami
#  Minimalni overlap - zvýrazni hranice
#  Může vyrešit problém prekrývajicich se trid
#  Použiti: klasifikace, feature selection

# ⭐ Kombinace PCA + LDA:
#  1. PCA: komprese (1000 → 100 features)
#  2. LDA: separace (100 → 10 features)
